In [2]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import re
import pandas
import time
import os
import xlwt
from openpyxl import load_workbook
from shutil import move
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait

In [11]:
url="https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page={}&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_{}"
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument('headless')
options.add_argument('Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like Mac OS X) AppleWebKit/604.3.5 (KHTML, like Gecko) Version/13.0 MQQBrowser/10.1.1 Mobile/15B87 Safari/604.1 QBWebViewUA/2 QBWebViewType/1 WKType/1')
options.add_experimental_option('excludeSwitches', ['enable-automation'])
chrome = webdriver.Chrome('./chromedriver', options=options)
amazon_allpage = []
data = []
tStart = time.time()
for i in range(1,3):
    chrome.get(url.format(i,i))
    print(url.format(i,i))
    soup = BeautifulSoup(chrome.page_source,'lxml')
    amazon_card=soup.find_all(class_="a-size-medium a-color-base a-text-normal")
    if len(amazon_card) == 0:
        print("找不到顯示卡標題")
        break
    data.append([ele.text for ele in amazon_card]) 
chrome.close()    
tEnd = time.time()
print("總共耗時:%f 秒" % (tEnd-tStart))

https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=1&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_1
https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=2&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_2
總共耗時:8.251323 秒


In [56]:
amazon_json = json.load(open("AMAZON_COMPUTER_PRICE" + ".json",encoding="utf-8"))
print(amazon_json['title'][2])

EVGA


In [52]:
title=[]
for adx,datas in enumerate(data):
    df = 'df'+str(adx) 
    df = pd.DataFrame(datas,columns=['f'])
    columns_of_datas = [ddd for ddd in range(0,len(datas))]  
    # 分種類並寫入json中
    for aaa in columns_of_datas:
    # 正則表達式將標題的牌子切割出來
        matchObj = re.split(r'\s', df['f'][aaa], re.M|re.I)
        title.append(matchObj[0])
# 篩選重複的牌子        
json_title_dumps = list(set(title))
json_title = {'title': json_title_dumps}
amazon_json.update(json_title)
# 將整理好title的json寫入json檔案中
with open('AMAZON_COMPUTER_PRICE.json', 'w', encoding='utf-8') as f:
    json.dump(amazon_json, f, ensure_ascii=False, indent=4,sort_keys=True)
print(amazon_json)

{'sheet_name': ['顯示卡', '第二頁'], 'title': ['MSI微星', 'ZOTAC', 'EVGA', 'Sapphire', 'MSI', 'GIGABYTE', 'Gigabyte', 'NVIDIA', 'PowerColor', 'XFX']}


In [63]:
for adx,datas in enumerate(data):
    df = 'df'+str(adx) 
    df = pd.DataFrame(datas,columns=['f'])
    #   每頁資料的數量
    columns_of_datas = [ddd for ddd in range(0,len(datas))]  
    
    #   將每頁資料歸納種類
    for aaa in columns_of_datas:
        print(amazon_json['title'][aaa if aaa <= len(amazon_json['title'])-1])
#         if amazon_json['title'][len(amazon_json['title'])-1] in df['f'][aaa]:
#             print(df['f'][aaa]) 
           

SyntaxError: invalid syntax (<ipython-input-63-d0c944810621>, line 9)

In [36]:
now_date = datetime.strftime(datetime.now() , '%Y%m%d')
# 沒檔案的話則建立
output_file = now_date + "_amazon_graphics card.xlsx"
target_path = "E:\\python\\比價、受歡迎系統\\Amazon\\Amazon_graphics_card\\"
if os.path.exists(output_file) == False:
    writer = pd.ExcelWriter(output_file, engine='openpyxl')
    writer.book.create_sheet('default')  # 新增空sheet
else:  # 如果檔案存在，append
    book = load_workbook(output_file)
    writer = pd.ExcelWriter(output_file, engine='openpyxl')
    writer.book = book
# 假如excel已經有json的sheet_name則移除excel的sheet
if amazon_json['sheet_name'] in writer.book.sheetnames:
    writer.book.remove(writer.book[amazon_json['sheet_name']])


# 將資料寫入excel
for adx,datas in enumerate(data):
    df = 'df'+str(adx) 
    
    df = pd.DataFrame(datas)
    df.to_excel(writer, sheet_name=amazon_json['sheet_name'][adx], index=False)
    
if (len(writer.book.sheetnames) > 1):  # 若有其他sheet，刪除default sheet
    if 'default' in writer.book.sheetnames:
        writer.book.remove(writer.book['default'])    
writer.save()  


In [37]:
cwd = os.getcwd()
curr_file = cwd +"\\"+ output_file
target_file = target_path + output_file
try:
    move(curr_file, target_file)
    print(output_file + "\n移動檔案完畢.")
except Exception as e:
    print(output_file + "\n移動檔案失敗.")
    print(e.args)

20200907_amazon_graphics card.xlsx
移動檔案完畢.
